## Set up matching DF's

In [65]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import os

In [66]:
directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_toto = max(files, key=os.path.getmtime)

directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_unibet = max(files, key=os.path.getmtime)

In [67]:
latest_file_unibet

'/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet/unibetAllSport2025-01-28 18:46:13.521238.csv'

In [68]:
toto_raw = pd.read_csv(f"{latest_file_toto}", index_col=0).drop_duplicates()
kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League', 'Italië Coppa Italia Vrouwen',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'England FA Cup Women', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)', 'Coppa Italia (D)', 'Liga MX Femenil'

]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3696476550.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3696476550.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3696476550.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [69]:
kambi_raw

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name
0,2489074265,1001159633,Totaal Aantal Doelpunten door Raith Rovers,Total Goals by Raith Rovers,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022265838,3625050445,Meer dan,Over,1280.0,500.0,NaN,OT_OVER,2025-01-27T09:25:03Z,7/25,-360.0,OPEN,ENABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship
1,2489074265,1001159633,Totaal Aantal Doelpunten door Raith Rovers,Total Goals by Raith Rovers,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022265838,3625050446,Minder dan,Under,3200.0,500.0,NaN,OT_UNDER,2025-01-27T09:25:03Z,11/5,220.0,OPEN,ENABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship
2,2489074266,1001159633,Totaal Aantal Doelpunten door Raith Rovers,Total Goals by Raith Rovers,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022265838,3625050447,Meer dan,Over,14000.0,3500.0,NaN,OT_OVER,2025-01-27T09:25:02Z,13/1,1300.0,OPEN,ENABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship
3,2489074266,1001159633,Totaal Aantal Doelpunten door Raith Rovers,Total Goals by Raith Rovers,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022265838,3625050448,Minder dan,Under,1020.0,3500.0,NaN,OT_UNDER,2025-01-27T09:25:02Z,1/50,-5000.0,OPEN,ENABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship
4,2489074268,1002244276,Asian Totaal,Asian Total,GOALS,FULL_TIME,21,Asian Over/Under,Asian Over/Under,1022265838,3625050451,Meer dan,Over,3650.0,3500.0,NaN,OT_OVER,2025-01-27T09:25:03Z,13/5,265.0,OPEN,DISABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123721,2485239037,1005190459,Draw No Bet - Gevechtsnoteringen,Draw No Bet - Bout Odds,NaN,NaN,2,Wedstrijd,Match,1022530769,3611783103,"Ball, Nick","Ball, Nick",1090.0,NaN,NaN,1,2025-01-11T16:31:29Z,1/12,-1115.0,OPEN,ENABLED,NaN,NaN,Nick Ball vs TJ Doheny,BOXING,Komende wedstrijden
123722,2485239037,1005190459,Draw No Bet - Gevechtsnoteringen,Draw No Bet - Bout Odds,NaN,NaN,2,Wedstrijd,Match,1022530769,3611783105,"Doheny, TJ","Doheny, TJ",7500.0,NaN,NaN,2,2025-01-11T16:31:29Z,13/2,650.0,OPEN,ENABLED,NaN,NaN,Nick Ball vs TJ Doheny,BOXING,Komende wedstrijden
123723,2485239038,1001159900,Wedstrijdnotering,Bout Odds,NaN,NaN,2,Wedstrijd,Match,1022530769,3611783106,"Ball, Nick","Ball, Nick",1100.0,NaN,NaN,1,2025-01-11T16:31:29Z,1/10,-1000.0,OPEN,ENABLED,NaN,NaN,Nick Ball vs TJ Doheny,BOXING,Komende wedstrijden
123724,2485239038,1001159900,Wedstrijdnotering,Bout Odds,NaN,NaN,2,Wedstrijd,Match,1022530769,3611783107,Gelijkspel,Draw,23000.0,NaN,NaN,OT_CROSS,2025-01-11T16:31:29Z,22/1,2200.0,OPEN,ENABLED,NaN,NaN,Nick Ball vs TJ Doheny,BOXING,Komende wedstrijden


Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [70]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_filtered_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

## Winnaar: Draw no Bet

In [71]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [72]:
# import unicodedata

# # Replace '-' with a space and remove accents in the 'names' column
# filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )
# # Replace '-' with a space and remove accents in the 'names' column
# filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
#     lambda x: ''.join(
#         char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
#         if not unicodedata.combining(char)
#     )
# )

# # Create 'Team1' and 'Team2'
# filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
# filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

# kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score, _ = result
#     return match

# # Apply matching function to toto_filtered_football
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

In [73]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip().replace('Athletic', '').replace('Atletico', '')
        team2 = parts[1].strip().replace('Athletic', '').replace('Atletico', '')

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/1566042253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/1566042253.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/1566042253.py:20: SettingW

,Event Name,matched_event
event_id,,
6667819,Bolton Wanderers vs Northampton Town,Bolton vs Northampton Town
6667819,Bolton Wanderers vs Northampton Town,Bolton vs Northampton Town
6667820,Burton Albion vs Reading,Burton Albion vs Reading
6667820,Burton Albion vs Reading,Burton Albion vs Reading
6667822,Charlton Athletic vs Bristol Rovers,Charlton Athletic vs Bristol Rovers


In [74]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,sex,Team1,Team2
31,2489074284,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022265838,3625050481,1,1,1.93,NaN,Hamilton Academical,1,2025-01-27T09:25:03Z,10/11,-108.0,OPEN,DISABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship,M,Hamilton Academical,Raith Rovers
32,2489074284,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022265838,3625050485,2,2,1.75,NaN,Raith Rovers,2,2025-01-27T09:25:03Z,3/4,-134.0,OPEN,DISABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship,M,Hamilton Academical,Raith Rovers
51,2489074302,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022265838,3625050521,1,1,2.00,NaN,Hamilton Academical,1,2025-01-27T09:25:02Z,Evens,100.0,OPEN,DISABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship,M,Hamilton Academical,Raith Rovers
52,2489074302,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022265838,3625050526,2,2,1.77,NaN,Raith Rovers,2,2025-01-27T09:25:02Z,3/4,-130.0,OPEN,DISABLED,NaN,NaN,Hamilton Academical vs Raith Rovers,FOOTBALL,Championship,M,Hamilton Academical,Raith Rovers
285,2489074431,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021104376,3625051019,1,1,1.41,NaN,Falkirk,1,2025-01-27T09:26:19Z,2/5,-245.0,OPEN,DISABLED,NaN,NaN,Falkirk vs Livingston,FOOTBALL,Championship,M,Falkirk,Livingston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123395,2489161578,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021622288,3625264511,2,2,2.43,NaN,Melbourne Victory (D),2,2025-01-27T15:59:00Z,7/5,143.0,OPEN,DISABLED,NaN,NaN,Melbourne City (W) vs Melbourne Victory (W),FOOTBALL,A-League (D),W,Melbourne City (W),Melbourne Victory (W)
123471,2489149737,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021622290,3625272894,1,1,1.28,NaN,Canberra United (D),1,2025-01-27T15:59:04Z,7/25,-360.0,OPEN,DISABLED,NaN,NaN,Canberra United (W) vs Perth Glory (W),FOOTBALL,A-League (D),W,Canberra United (W),Perth Glory (W)
123472,2489149737,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021622290,3625272898,2,2,3.35,NaN,Perth Glory (D),2,2025-01-27T15:59:04Z,47/20,235.0,OPEN,DISABLED,NaN,NaN,Canberra United (W) vs Perth Glory (W),FOOTBALL,A-League (D),W,Canberra United (W),Perth Glory (W)
123498,2489126757,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021622279,3625329785,1,1,1.33,NaN,Brisbane Roar FC (D),1,2025-01-27T15:59:08Z,33/100,-305.0,OPEN,DISABLED,NaN,NaN,Brisbane Roar FC (W) vs Wellington Phoenix FC (W),FOOTBALL,A-League (D),W,Brisbane Roar FC (W),Wellington Phoenix FC (W)


In [75]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# Display the result
matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [76]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [77]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex'], 
    right_on=['event_name', 'standardized_label', 'sex'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

## Over/Under

In [78]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [79]:
kambi_filtered_football_overunder = kambi_filtered_football[kambi_filtered_football['bet_offer_type_name'].str.contains('Over')]
toto_filtered_football_overunder = toto_filtered_football[(toto_filtered_football['Outcome Name'].str.contains('Over')) | (toto_filtered_football['Outcome Name'].str.contains('Under'))]

In [80]:
kambi_filtered_football_overunder.event_name.unique()

array(['Hamilton Academical vs Raith Rovers', 'Falkirk vs Livingston',
       'Dunfermline Athletic vs Raith Rovers',
       'Hamilton Academical vs Ayr United',
       'Partick Thistle vs Greenock Morton',
       "Airdrieonians vs Queen's Park", 'Dundee FC vs Hearts',
       'Hibernian vs Aberdeen', 'St Mirren vs St Johnstone',
       'Kilmarnock vs Dundee United', 'Motherwell vs Celtic',
       'Rangers FC vs Ross County', 'Boavista vs Famalicão',
       'Santa Clara vs Casa Pia', 'Nacional Madeira vs Arouca',
       'Gil Vicente vs Estoril', 'Vitoria Guimarães vs AVS Futebol Sad',
       'Sporting Lisbon vs Farense', 'CF Estrela vs Benfica',
       'Moreirense vs S.C. Braga', 'Rio Ave vs FC Porto',
       'Çorum Belediyespor vs Bandirmaspor', 'Konyaspor vs Bodrumspor',
       'Eyüpspor vs Sivasspor', 'Hatayspor vs Antalyaspor A.Ş.',
       'Kayserispor vs Trabzonspor A.Ş.',
       'İstanbul Başakşehir vs Samsunspor',
       'Adana Demirspor vs Kasımpaşa', 'Göztepe A.Ş. vs Alanyaspor

In [81]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [82]:
# Get unique records from 'Event Name' and 'matched_event'
unique_records = toto_filtered_football_overunder[['Event Name', 'matched_event']].drop_duplicates()

# Display the result
unique_records[~unique_records['matched_event'].isna()]

,Event Name,matched_event
0,Bolton Wanderers vs Northampton Town,Bolton vs Northampton Town
6,Burton Albion vs Reading,Burton Albion vs Reading
12,Charlton Athletic vs Bristol Rovers,Charlton Athletic vs Bristol Rovers
18,Exeter City vs Leyton Orient FC,Exeter vs Leyton Orient
24,Lincoln City vs Blackpool FC,Lincoln City vs Blackpool
...,...,...
6503,Western United FC vs Western Sydney Wanderers,Western United (W) vs Western Sydney Wanderers...
6601,Rio Claro FC SP vs Capivariano FC SP,Rio Claro SP vs Capivariano SP
6679,Tombense FC MG vs Pouso Alegre MG,Tombense MG vs Pouso Alegre MG
6737,SC Aymores MG vs Uberlandia EC MG,SC Aymores vs Uberlandia MG


In [83]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [84]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: 'Goals' if 'Doelpunten' in x else 'Other'
)

# Create 'OverUnderTime'
def determine_over_under_time(label):
    if '1e helft' in label.lower():
        return '1e Helft'
    elif '2e helft' in label.lower():
        return '2e Helft'
    elif label[-2:].isdigit():  # Check if the last two characters in the label are digits
        return label.split(' - ')[-1]
    else:
        return 'Full Time'

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
        '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3155310470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_57829/3155310470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [85]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if 'Goals' in x else 'Other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else ('2e Helft' if '2e helft' in x.lower() else 'Full Time')
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['Market Name'] else (
        '2' if row['Team2'] in row['Market Name'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# Create 'line'
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: float(x.split('Over/Under ')[-1]) if 'Over/Under ' in x else None
)

In [86]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)
# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name']]

# Display the merged DataFrame
merged_football_overunder.head()

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
0,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,Under,1.33,33,100,HL,L,Voetbal,Engeland League 1,Burton Albion vs Reading,Burton Albion,Reading,M,Burton Albion vs Reading,100.0,Goals,Full Time,Burton Albion,Reading,Total team 1 and team 2,3.5,2487691667,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097078,3620306738,Meer dan,Over,2.95,NaN,OT_OVER,2025-01-28T17:29:36Z,39/20,195.0,OPEN,ENABLED,NaN,NaN,Burton Albion vs Reading,FOOTBALL,League 1,Burton Albion,Reading
3,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,Over,3.00,2,1,HL,1,Voetbal,Engeland League 1,Burton Albion vs Reading,Burton Albion,Reading,M,Burton Albion vs Reading,100.0,Goals,Full Time,Burton Albion,Reading,Total team 1 and team 2,3.5,2487691667,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097078,3620306745,Minder dan,Under,1.35,NaN,OT_UNDER,2025-01-28T17:29:36Z,7/20,-286.0,OPEN,ENABLED,NaN,NaN,Burton Albion vs Reading,FOOTBALL,League 1,Burton Albion,Reading
5,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,Over,2.90,19,10,HL,1,Voetbal,Engeland League 1,Charlton Athletic vs Bristol Rovers,Charlton Athletic,Bristol Rovers,M,Charlton Athletic vs Bristol Rovers,100.0,Goals,Full Time,Charlton Athletic,Bristol Rovers,Total team 1 and team 2,3.5,2487704743,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097071,3620294144,Minder dan,Under,1.32,NaN,OT_UNDER,2025-01-28T18:42:44Z,8/25,-315.0,OPEN,ENABLED,NaN,NaN,Charlton Athletic vs Bristol Rovers,FOOTBALL,League 1,Charlton Athletic,Bristol Rovers
6,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,Under,1.36,9,25,HL,L,Voetbal,Engeland League 1,Charlton Athletic vs Bristol Rovers,Charlton Athletic,Bristol Rovers,M,Charlton Athletic vs Bristol Rovers,100.0,Goals,Full Time,Charlton Athletic,Bristol Rovers,Total team 1 and team 2,3.5,2487704743,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097071,3620294142,Meer dan,Over,3.15,NaN,OT_OVER,2025-01-28T18:42:44Z,43/20,215.0,OPEN,ENABLED,NaN,NaN,Charlton Athletic vs Bristol Rovers,FOOTBALL,League 1,Charlton Athletic,Bristol Rovers
9,Exeter City vs Leyton Orient FC,Aantal Goals Over/Under 3.5,Over,3.75,11,4,HL,1,Voetbal,Engeland League 1,Exeter City vs Leyton Orient FC,Exeter City,Leyton Orient FC,M,Exeter vs Leyton Orient,100.0,Goals,Full Time,Exeter City,Leyton Orient FC,Total team 1 and team 2,3.5,2487708558,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097068,3620281499,Minder dan,Under,1.22,NaN,OT_UNDER,2025-01-28T13:40:20Z,11/50,-455.0,OPEN,ENABLED,NaN,NaN,Exeter vs Leyton Orient,FOOTBALL,League 1,Exeter,Leyton Orient


## Match Winnaar & OverUnder and do Arb Analysis

In [87]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,group_name,home_score,home_team,lifetime,line,match_name,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,status,type
0,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,1.33,Under,L,HL,Full Time,Goals,Total team 1 and team 2,100,33,Burton Albion,Burton Albion,Reading,Reading,NaN,Reading,2.487692e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-28T17:29:36Z,Engeland League 1,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021097e+09,Burton Albion vs Reading,100.0,League 1,NaN,Burton Albion,FULL_TIME,3.5,Burton Albion vs Reading,Burton Albion vs Reading,GOALS,2.95,195.0,39/20,Over,3.620307e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
1,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,3.00,Over,1,HL,Full Time,Goals,Total team 1 and team 2,1,2,Burton Albion,Burton Albion,Reading,Reading,NaN,Reading,2.487692e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-28T17:29:36Z,Engeland League 1,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021097e+09,Burton Albion vs Reading,100.0,League 1,NaN,Burton Albion,FULL_TIME,3.5,Burton Albion vs Reading,Burton Albion vs Reading,GOALS,1.35,-286.0,7/20,Under,3.620307e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
2,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,2.90,Over,1,HL,Full Time,Goals,Total team 1 and team 2,10,19,Charlton Athletic,Charlton Athletic,Bristol Rovers,Bristol Rovers,NaN,Bristol Rovers,2.487705e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-28T18:42:44Z,Engeland League 1,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021097e+09,Charlton Athletic vs Bristol Rovers,100.0,League 1,NaN,Charlton Athletic,FULL_TIME,3.5,Charlton Athletic vs Bristol Rovers,Charlton Athletic vs Bristol Rovers,GOALS,1.32,-315.0,8/25,Under,3.620294e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
3,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,1.36,Under,L,HL,Full Time,Goals,Total team 1 and team 2,25,9,Charlton Athletic,Charlton Athletic,Bristol Rovers,Bristol Rovers,NaN,Bristol Rovers,2.487705e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-28T18:42:44Z,Engeland League 1,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021097e+09,Charlton Athletic vs Bristol Rovers,100.0,League 1,NaN,Charlton Athletic,FULL_TIME,3.5,Charlton Athletic vs Bristol Rovers,Charlton Athletic vs Bristol Rovers,GOALS,3.15,215.0,43/20,Over,3.620294e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_OVER
4,Exeter City vs Leyton Orient FC,Aantal Goals Over/Under 3.5,3.75,Over,1,HL,Full Time,Goals,Total team 1 and team 2,4,11,Exeter City,Exeter,Leyton Orient FC,Leyton Orient,NaN,Leyton Orient FC,2.487709e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-01-28T13:40:20Z,Engeland League 1,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021097e+09,Exeter vs Leyton Orient,100.0,League 1,NaN,Exeter City,FULL_TIME,3.5,Exeter City vs Leyton Orient FC,Exeter vs Leyton Orient,GOALS,1.22,-455.0,11/50,Under,3.620281e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,OPEN,OT_UNDER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,Fiorentina FC vs AC Milan,Draw No Bet,2.10,AC Milan,2,DN,NaN,NaN,NaN,10,11,Fiorentina FC,Fiorentina (W),AC Milan,Milan (W),NaN,AC Milan,2.489203e+09,Match,2.0,Wedstrijd,DISABLED,2025-01-28T16:37:57Z,Italië Coppa Italia Vrouwen,Draw No Bet,1.001160e+09,Draw No Bet,1.022613e+09,Fioren

In [88]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,Under,Meer dan,1.33,2.95,109.086275,False,0,0
1,Burton Albion vs Reading,Aantal Goals Over/Under 3.5,Over,Minder dan,3.00,1.35,107.407407,False,0,0
2,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,Over,Minder dan,2.90,1.32,110.240334,False,0,0
3,Charlton Athletic vs Bristol Rovers,Aantal Goals Over/Under 3.5,Under,Meer dan,1.36,3.15,105.275444,False,0,0
4,Exeter City vs Leyton Orient FC,Aantal Goals Over/Under 3.5,Over,Minder dan,3.75,1.22,108.633880,False,0,0
...,...,...,...,...,...,...,...,...,...,...
1403,Fiorentina FC vs AC Milan,Draw No Bet,AC Milan,1,2.10,1.66,107.860011,False,0,0
1404,Fiorentina FC vs AC Milan,2e Helft: Draw No Bet,Fiorentina FC,NaN,1.61,NaN,NaN,False,0,0
1405,Fiorentina FC vs AC Milan,2e Helft: Draw No Bet,AC Milan,NaN,2.00,NaN,NaN,False,0,0
1406,Fiorentina FC vs AC Milan,1e Helft: Draw No Bet,Fiorentina FC,NaN,1.59,NaN,NaN,False,0,0


In [89]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [90]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'outcome_label', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,outcome_label,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex
